In [2]:
!pip install datasets evaluate sacrebleu git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to c:\users\markv\appdata\local\temp\pip-req-build-mu8ah7df
  Resolved https://github.com/openai/whisper.git to commit 5979f03701209bb035a0a466f14131aeb1116cbb
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\markv\AppData\Local\Temp\pip-req-build-mu8ah7df'


In [1]:
!pip install transformers==4.45.2

In [7]:
!pip install protobuf==3.20.0

   ---------------------------------------- 0.0/903.8 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/903.8 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 903.8/903.8 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.


In [1]:
# Required imports
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, DatasetDict
import evaluate

c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the Tydi XOR dataset from Hugging Face and filter for languages
dataset = load_dataset("coastalcph/tydi_xor_rc")

# Filter the dataset for Finnish, Japanese, and Russian languages
languages = ['fi', 'ja', 'ru']
train_data = dataset["train"].filter(lambda x: x['lang'] in languages)
valid_data = dataset["validation"].filter(lambda x: x['lang'] in languages)

In [4]:
# Instantiate tokenizer and model for mBART (multilingual)
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [3]:
# If you want to load model from file
model_path = "./resultsGpu/checkpoint-500"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBart50TokenizerFast.from_pretrained(model_path)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)

Using device: cuda


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [5]:
# Preprocessing function for tokenization
def preprocess_data(batch):
    inputs = []
    targets = []

    # For each question-context-answer pair
    for question, context, lang, answer, answerable in zip(
        batch["question"], batch["context"], batch["lang"], batch["answer"], batch["answerable"]
    ):
        # Construct input as "question: <question> context: <context>"
        input_text = f"question: {question} context: {context}"
        inputs.append(input_text)

        # For answerable cases, we use the actual answer; otherwise, we use "No answer"
        target_text = answer if answerable else "No answer"
        targets.append(target_text)

    # Tokenize input and target pairs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the train and validation sets
tokenized_train = train_data.map(preprocess_data, batched=True)
tokenized_valid = valid_data.map(preprocess_data, batched=True)

Map: 100%|██████████| 1380/1380 [00:00<00:00, 2192.67 examples/s]


In [6]:
# Define metric for evaluation
bleu_metric = evaluate.load("sacrebleu")

# Custom evaluation function to compute BLEU scores
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU score
    bleu = bleu_metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    return {"bleu": bleu["score"]}

In [36]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./resultsGpu",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train.select(range(1000)),
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

  1%|▏         | 10/750 [05:36<5:36:20, 27.27s/it]

{'loss': 9.9309, 'grad_norm': 99.59172821044922, 'learning_rate': 2.976e-05, 'epoch': 0.04}


  3%|▎         | 20/750 [13:29<7:40:53, 37.88s/it]

{'loss': 8.162, 'grad_norm': 104.07878112792969, 'learning_rate': 2.936e-05, 'epoch': 0.08}


  4%|▍         | 30/750 [18:23<5:28:01, 27.33s/it]

...

{'loss': 0.0032, 'grad_norm': 0.3601582944393158, 'learning_rate': 5.36e-06, 'epoch': 2.48}


 84%|████████▍ | 630/750 [10:51:26<41:37, 20.81s/it]

{'loss': 0.0037, 'grad_norm': 0.35988935828208923, 'learning_rate': 4.96e-06, 'epoch': 2.52}


 84%|████████▍ | 632/750 [10:52:03<40:54, 20.80s/it]

KeyboardInterrupt: 

In [7]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./resultsGpu",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train.select(range(4)),
    eval_dataset=tokenized_valid.select(range(4)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

c:\Users\markv\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                             
 33%|███▎      | 1/3 [01:20<00:59, 29.87s/it]

{'eval_loss': 0.04223838075995445, 'eval_bleu': 0.0, 'eval_runtime': 50.2783, 'eval_samples_per_second': 0.08, 'eval_steps_per_second': 0.02, 'epoch': 1.0}


                                             
 67%|██████▋   | 2/3 [02:57<01:06, 66.27s/it]

{'eval_loss': 0.04363738372921944, 'eval_bleu': 0.0, 'eval_runtime': 55.5094, 'eval_samples_per_second': 0.072, 'eval_steps_per_second': 0.018, 'epoch': 2.0}


                                             
100%|██████████| 3/3 [04:21<00:00, 87.18s/it]

{'eval_loss': 0.044128358364105225, 'eval_bleu': 0.0, 'eval_runtime': 28.613, 'eval_samples_per_second': 0.14, 'eval_steps_per_second': 0.035, 'epoch': 3.0}
{'train_runtime': 261.5623, 'train_samples_per_second': 0.046, 'train_steps_per_second': 0.011, 'train_loss': 0.0005929085503642758, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=0.0005929085503642758, metrics={'train_runtime': 261.5623, 'train_samples_per_second': 0.046, 'train_steps_per_second': 0.011, 'total_flos': 13002778607616.0, 'train_loss': 0.0005929085503642758, 'epoch': 3.0})

In [47]:
# Evaluate and collect results on validation set
eval_results = trainer.evaluate(tokenized_valid.select(range(8)))

# Separate the validation dataset into answerable and unanswerable subsets
answerable_data = tokenized_valid.filter(lambda x: x["answerable"]).select(range(8))
unanswerable_data = tokenized_valid.filter(lambda x: not x["answerable"]).select(range(8))

# Evaluate on answerable examples
answerable_results = trainer.evaluate(answerable_data)

# Evaluate on unanswerable examples
unanswerable_results = trainer.evaluate(unanswerable_data)

# Print overall results and answerable/unanswerable breakdown
print("Overall BLEU:", eval_results["eval_bleu"])
print("Answerable BLEU:", answerable_results["eval_bleu"])
print("Unanswerable BLEU:", unanswerable_results["eval_bleu"])

                                                    
 84%|████████▍ | 632/750 [11:07:37<40:54, 20.80s/it]

{'eval_loss': 0.05551483482122421, 'eval_bleu': 4.159883186137868, 'eval_runtime': 59.0177, 'eval_samples_per_second': 0.136, 'eval_steps_per_second': 0.034, 'epoch': 2.53}


Filter: 100%|██████████| 1380/1380 [00:00<00:00, 2036.29 examples/s]
                                                    
 84%|████████▍ | 632/750 [11:08:38<40:54, 20.80s/it]

{'eval_loss': 0.05551483482122421, 'eval_bleu': 4.159883186137868, 'eval_runtime': 58.9688, 'eval_samples_per_second': 0.136, 'eval_steps_per_second': 0.034, 'epoch': 2.53}


                                                    
 84%|████████▍ | 632/750 [11:09:40<40:54, 20.80s/it]

{'eval_loss': 0.0026778103783726692, 'eval_bleu': 0.0, 'eval_runtime': 62.0308, 'eval_samples_per_second': 0.129, 'eval_steps_per_second': 0.032, 'epoch': 2.53}
Overall BLEU: 4.159883186137868
Answerable BLEU: 4.159883186137868
Unanswerable BLEU: 0.0


In [39]:
# Sample input for prediction
sample_question = tokenized_valid.select(range(2,3))['question'][0]
sample_context = tokenized_valid.select(range(2,3))['context'][0]

# Construct the input text
input_text = f"question: {sample_question} context: {sample_context}"

# Tokenize the input
input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids.to(device)

# Generate the answer
with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)

# Decode the generated output
generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated answer
print(f"Question: {sample_question}")
print(f"Context: {sample_context}")
print(f"Generated Answer: {generated_answer}")


Question: Kuka oli Glee sarjan pääosassa?
Context: Rachel Barbra Berry (Lea Michele) is the lead character and is a "strong, driven" member of the glee club, who is misunderstood by her peers. Michele took the role in "Glee" because of Rachel's characterization, explaining: "Not only is she a singer, but she has so much heart—I think it's what we need on TV." Michele described the first thirteen episodes of the series as: "Rachel's journey of finding herself within the glee club", explaining that: "She's learning how to be a team player and work within this group."
Generated Answer: Rachel Barbra Berry


In [30]:
print(tokenized_train.select(range(1,2))['question'][0])
print(tokenized_train.select(range(1,2))['context'])
print(tokenized_train.select(range(1,2))['answer'])
print(tokenized_train.select(range(1,2))['answerable'])

ビスカヤ県で初めて進出した大規模鉱業会社は何？
['Another consequence of the Carlist defeat and ensuing abolition of the Basque institutional system was the Liberalization of the industries on the Basque Provinces, especially in Biscay. The liberalization of the mines, industries and ports attracted many companies, specially British Mining Companies, that established in Biscay along with small local societies, such as Ybarra-Mier y Compañía, creating a big industrial society, based on iron mining and industry. These expansion created very big mining companies, such as Orconera Iron Ore Company Limited and Societé Franco-Belge des Mines de Somorrostro.']
['Ybarra-Mier y Compañía']
[True]


In [12]:
# Sample input for prediction
sample_question = tokenized_train.select(range(1,2))['question'][0]
sample_context = tokenized_train.select(range(1,2))['context'][0]

# Construct the input text
input_text = f"question: {sample_question} context: {sample_context}"

# Tokenize the input
input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids.to(device)

# Generate the answer
with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)

# Decode the generated output
generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated answer
print(f"Question: {sample_question}")
print(f"Context: {sample_context}")
print(f"Generated Answer: {generated_answer}")


Question: ビスカヤ県で初めて進出した大規模鉱業会社は何？
Context: Another consequence of the Carlist defeat and ensuing abolition of the Basque institutional system was the Liberalization of the industries on the Basque Provinces, especially in Biscay. The liberalization of the mines, industries and ports attracted many companies, specially British Mining Companies, that established in Biscay along with small local societies, such as Ybarra-Mier y Compañía, creating a big industrial society, based on iron mining and industry. These expansion created very big mining companies, such as Orconera Iron Ore Company Limited and Societé Franco-Belge des Mines de Somorrostro.
Generated Answer: Ybarra-Mier y Compañía


In [43]:
model.save_pretrained('./resultsGpu/2.5epochs')